In [1]:
!pip install flask-ngrok
!pip install flask-bootstrap
!pip install pyngrok==4.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 456.4/456.4 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for flask-bootstrap: filename=Flask_Bootstrap-3.3.7.1-py3-none-any.whl size=460118 sha256=c644086f1daf47b210b1fd149dcbfa7a71e56132e5b739ef92e704e1fc731779
  Stored in directory: /root/.cache/pip/wheels/6f/33/ad/26540e84a28334e5dfeda756df270f95353779f03bc5cf40d4
  Created wheel for visitor: filename=visitor-0.1.3-py3-none-any.whl size=3926 sha256=9314cdf0a927cd9e8b10d7cd5cfa084e1c92d35a698e612960bfdf902e1db2f8
  Stored in directory: /root/.cache/pip/wheels/19/31/99/2ec5b4459cac4d801d6201d501a354366d180afc9f8bb2d333
Successfully built flask-bootstrap visitor
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=4769f32c3ac253b3f9db004234ca6e32f0bd1c7cfda8376270ccccd6beb99b7a
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2e

In [2]:
import pandas as pd
from flask_ngrok import run_with_ngrok
from flask import request, jsonify, Flask
import random as r

In [30]:
app = Flask(__name__) #the name of the application package
run_with_ngrok(app)

In [31]:
!ngrok authtoken 2Ywj10KuZ2rOwP2XBKQNI0tCFfq_2WWaUYFoCAb5Huf6P4oGe

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [19]:
from torchvision.models import resnet50, ResNet50_Weights
import joblib
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
joblib.dump(model, 'model.pkl')
print("Model dumped!")

Model dumped!


In [21]:
# Preprocess image
from torchvision import transforms
from PIL import Image
img = Image.open('img.jpg')
tfms = transforms.Compose([transforms.Resize(224), transforms.CenterCrop(224),
                           transforms.ToTensor(),
                           transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
img = tfms(img).unsqueeze(0)

In [69]:
import torch
import json
from flask import Flask, render_template
from PIL import Image
from io import BytesIO
import base64
import traceback

app = Flask(__name__,static_folder='static')
run_with_ngrok(app)

@app.route("/")
def hello():
    return render_template('hello.html')


@app.route('/input', methods=['GET', 'POST'])
def show_index():
    img_path = '/content/statics/img.jpg'
    with open(img_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read()).decode('utf-8')

    return render_template('index.html', img_data=encoded_string)

@app.route('/predict', methods=['GET','POST'])
def predict():
    if model:
        try:
            model.eval()
            with torch.no_grad():
                logits = model(img)
            preds = torch.topk(logits, k=5).indices.squeeze(0).tolist()
            labels_map = json.load(open('/content/labels_map.txt'))
            labels_map = [labels_map[str(i)] for i in range(1000)]
            output ={}
            for idx in preds:
                label = labels_map[idx]
                prob = torch.softmax(logits, dim=1)[0, idx].item()
                print('{:<75} ({:.2f}%)'.format(label, prob*100))
                output[label] = '({:.2f}%)'.format(prob*100)
            return jsonify({'prediction': str(output)})

        except:

            return jsonify({'trace': traceback.format_exc()})
    else:
        print ('Train the model first')
        return ('No model here to use')

if __name__ == '__main__':
    model = joblib.load("model.pkl") # Load "model.pkl"
    print ('Model loaded')

    app.run()

Model loaded
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://cdfa-104-197-162-30.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [17/Dec/2023 06:03:07] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Dec/2023 06:03:07] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [17/Dec/2023 06:03:26] "GET /input HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Dec/2023 06:03:32] "GET /predict HTTP/1.1" 200 -


giant panda, panda, panda bear, coon bear, Ailuropoda melanoleuca           (45.03%)
lesser panda, red panda, panda, bear cat, cat bear, Ailurus fulgens         (0.24%)
limousine, limo                                                             (0.20%)
soccer ball                                                                 (0.16%)
Walker hound, Walker foxhound                                               (0.16%)
